In [36]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
from torchvision import datasets, transforms
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from torchvision.utils import save_image

In [37]:
batch_size = 128
epochs = 10
seed = 1
log_interval = 10
T.manual_seed(seed)
device = T.device('cuda' if T.cuda.is_available() else 'cpu')
print('Device : ', device)

Device :  cuda


In [38]:
kwargs = {'num_workers': 1, 'pin_memory': True}

train_loader = T.utils.data.DataLoader(
    datasets.MNIST('data/', train=True, download=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = T.utils.data.DataLoader(
    datasets.MNIST('data/', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [39]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # encoder
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        # decoder
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = T.exp(0.5*logvar)
        eps = T.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return T.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [40]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [41]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * T.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [42]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with T.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = T.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results_note/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [43]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with T.no_grad():
        sample = T.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                    'results_note/sample_' + str(epoch) + '.png')

0/60000 (0%)]	Loss: 117.065254
Train Epoch: 3 [1280/60000 (2%)]	Loss: 114.811470
Train Epoch: 3 [2560/60000 (4%)]	Loss: 114.763657
Train Epoch: 3 [3840/60000 (6%)]	Loss: 119.434021
Train Epoch: 3 [5120/60000 (9%)]	Loss: 113.214653
Train Epoch: 3 [6400/60000 (11%)]	Loss: 119.106102
Train Epoch: 3 [7680/60000 (13%)]	Loss: 113.529953
Train Epoch: 3 [8960/60000 (15%)]	Loss: 113.439072
Train Epoch: 3 [10240/60000 (17%)]	Loss: 113.291817
Train Epoch: 3 [11520/60000 (19%)]	Loss: 116.415443
Train Epoch: 3 [12800/60000 (21%)]	Loss: 116.494415
Train Epoch: 3 [14080/60000 (23%)]	Loss: 115.286537
Train Epoch: 3 [15360/60000 (26%)]	Loss: 113.007622
Train Epoch: 3 [16640/60000 (28%)]	Loss: 116.642227
Train Epoch: 3 [17920/60000 (30%)]	Loss: 116.871040
Train Epoch: 3 [19200/60000 (32%)]	Loss: 109.764984
Train Epoch: 3 [20480/60000 (34%)]	Loss: 113.505577
Train Epoch: 3 [21760/60000 (36%)]	Loss: 113.196274
Train Epoch: 3 [23040/60000 (38%)]	Loss: 112.494019
Train Epoch: 3 [24320/60000 (41%)]	Loss: 112